In [17]:
import pandas as pd

In [18]:
pd.options.display.max_columns=21
voice = pd.read_csv('voice.csv')

In [19]:
# 将label转换成0和1，male 1; female 0
voice['label'] = voice['label'].apply(lambda x: 1 if x=='male' else 0)

In [20]:
from sklearn.metrics import classification_report

In [21]:
from deepctr.models import DeepFM, WDL
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [22]:
# generate feature columns
features = list(voice.columns)
dense_feature = features[:-1]

dense_feature_columns = [DenseFeat(feature, dimension=1) for feature in dense_feature]

linear_feature = dense_feature_columns
deep_feature = dense_feature_columns

In [23]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
voice[dense_feature] = ss.fit_transform(voice[dense_feature])

In [24]:
# data split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(voice[dense_feature], voice['label'], test_size=0.2, random_state=2020)

In [25]:
train_data = {name: x_train[name].values for name in dense_feature}
test_data = {name: x_test[name].values for name in dense_feature}

In [34]:
%%time
deepfm = DeepFM(linear_feature_columns=linear_feature,
                dnn_feature_columns=deep_feature,
                dnn_hidden_units=(32,32),
                seed=2020,
                task='binary')
deepfm.compile('adam', loss='BinaryCrossentropy' , metrics='acc')

# 查看可以使用什么loss
# import tensorflow as tf
# tf.keras.losses??

import multiprocessing
history = deepfm.fit(train_data, y_train, batch_size=64, epochs=110, verbose=True, validation_split=0.2, use_multiprocessing=multiprocessing.cpu_count())

Epoch 1/110
32/32 [==============================] - 0s 9ms/step - loss: 0.8325 - acc: 0.5935 - val_loss: 0.6322 - val_acc: 0.6805
Epoch 2/110
32/32 [==============================] - 0s 2ms/step - loss: 0.4668 - acc: 0.7908 - val_loss: 0.3781 - val_acc: 0.8698
Epoch 3/110
32/32 [==============================] - 0s 2ms/step - loss: 0.3123 - acc: 0.8984 - val_loss: 0.2763 - val_acc: 0.9211
Epoch 4/110
32/32 [==============================] - 0s 2ms/step - loss: 0.2324 - acc: 0.9255 - val_loss: 0.2108 - val_acc: 0.9487
Epoch 5/110
32/32 [==============================] - 0s 2ms/step - loss: 0.1777 - acc: 0.9433 - val_loss: 0.1645 - val_acc: 0.9546
Epoch 6/110
32/32 [==============================] - 0s 2ms/step - loss: 0.1403 - acc: 0.9526 - val_loss: 0.1353 - val_acc: 0.9625
Epoch 7/110
32/32 [==============================] - 0s 2ms/step - loss: 0.1179 - acc: 0.9581 - val_loss: 0.1153 - val_acc: 0.9724
Epoch 8/110
32/32 [==============================] - 0s 2ms/step - loss: 0.1039 - a

In [35]:
pred = deepfm.predict(test_data)
import numpy as np
from sklearn.metrics import classification_report
print(classification_report(y_test, np.where(pred>0.5, 1,0), digits=4))

              precision    recall  f1-score   support

           0     0.9870    0.9870    0.9870       308
           1     0.9877    0.9877    0.9877       326

    accuracy                         0.9874       634
   macro avg     0.9874    0.9874    0.9874       634
weighted avg     0.9874    0.9874    0.9874       634

